In [30]:
%%sh

# Specify an algorithm name
algorithm_name=whisper-fastapi

account=$(aws sts get-caller-identity --query Account --output text)

# Get the region defined in the current configuration (default to us-west-2 if none defined)
region=$(aws configure get region)
#region=${region:-us-west-2}

fullname="${account}.dkr.ecr.${region}.amazonaws.com/${algorithm_name}:latest"

# If the repository doesn't exist in ECR, create it.

aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
if [ $? -ne 0 ]
then
aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
fi

# Get the login command from ECR and execute it directly

aws ecr get-login-password --region ${region}|docker login --username AWS --password-stdin ${fullname}

# Build the docker image locally with the image name and then push it to ECR
# with the full name.

docker build -t ${algorithm_name} .
docker tag ${algorithm_name} ${fullname}

docker push ${fullname}

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
 credential helper to remove this warning. See
login/#credentials-storeengine/reference/commandline/



Login Succeeded


#0 building with "default" instance using docker driver

ckerfilernal] load build definition from Do
#1 transferring dockerfile: 2B done
#1 DONE 0.0s
ERROR: failed to solve: failed to read dockerfile: open Dockerfile: no such file or directory


The push refers to repository [707684582322.dkr.ecr.us-west-2.amazonaws.com/whisper-fastapi]
48c4b7350964: Preparing
8aba38bb6197: Preparing
81188348b0fc: Preparing
6084d2eb16c5: Preparing
a0787ac4: Preparing
eb7adf68ecdc: Preparing
339a848ed91d: Preparing
41158a36491b: Preparing
b079d8155d35: Preparing
c0b7ea454710: Preparing
08a626f2d039: Preparing
af35a154b011: Preparing
d60df4bdcba5: Preparing
cb2ff96c: Preparing
2beb086414c7: Preparing
7bf0aff327dd: Preparing
bcd30552e08a: Preparing
a3c7: Preparing
aded6b31b730: Preparing
44a5b21fc98e: Preparing
bbdba4451161: Preparing
: Preparing8
b0dfaf1ca5c5: Preparing
700fe921ad1f: Preparing
520e0f301880: Preparing
eparing8d6e0: Pr
e942261d196e: Preparing
eeb5315df33c: Preparing
022bf74291b2: Preparing
ing407952594: Prepar
5498e8c22f69: Preparing
eb7adf68ecdc: Waiting
339a848ed91d: Waiting
44a5b21fc98e: Waiting
451161: Waiting
41158a36491b: Waiting
4e356886a648: Waiting
b079d8155d35: Waiting
ingfaf1ca5c5: Wait
700fe921ad1f: Waiting
c0b7ea45471

In [31]:
import boto3
import json
import time

# Set your ECR image URI
image_uri = "707684582322.dkr.ecr.us-west-2.amazonaws.com/whisper-fastapi"

In [32]:
# Initialize boto3 clients
session = boto3.Session()
sm_client = session.client('sagemaker')

# Get the SageMaker execution role from the notebook environment
import sagemaker
role_arn = sagemaker.get_execution_role()
print(f"Using SageMaker execution role: {role_arn}")

[05/22/25 15:02:15] INFO     Found credentials from IAM Role:                                   ]8;id=273245;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=610030;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

Using SageMaker execution role: arn:aws:iam::707684582322:role/service-role/AmazonSageMaker-ExecutionRole-20191024T163188


In [33]:
model_name = "whisper"
instance_type = "ml.g5.2xlarge"

In [34]:
# Update your container definition with environment variables
container = {
    'Image': image_uri,  # Should be full ECR URI (e.g., account.dkr.ecr.region.amazonaws.com/whisper-fastapi:latest)
    'Mode': 'SingleModel',
    'Environment': {
        'HUGGING_FACE_HUB_TOKEN': 'hf_YWAymmnG*******8',  # Your HF token
    }
}

# Create the model with the container
model_response = sm_client.create_model(
    ModelName=model_name,
    Containers=[container],  # Use `Containers` instead of `PrimaryContainer`
    ExecutionRoleArn=role_arn
)

In [35]:
# Create endpoint config
sm_client.create_endpoint_config(
    EndpointConfigName='whisper-async-config',
    AsyncInferenceConfig={
        'OutputConfig': {
            'S3OutputPath': 's3://knowledge-base-us-west-2-707684582322/transcript/outputs'
        }
    },
    ProductionVariants=[
        {
            'VariantName': 'whisper-variant',
            'ModelName': model_name,
            'InstanceType': instance_type,  # GPU instance
            'InitialInstanceCount': 1
        }
    ]
)

# Create async endpoint
sm_client.create_endpoint(
    EndpointName='whisper-async-endpoint',
    EndpointConfigName='whisper-async-config'
)

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:707684582322:endpoint/whisper-async-endpoint',
 'ResponseMetadata': {'RequestId': '6a337c27-9ac5-4a32-a314-f1fced4725d2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '6a337c27-9ac5-4a32-a314-f1fced4725d2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Thu, 22 May 2025 15:02:22 GMT'},
  'RetryAttempts': 0}}

In [20]:
import json

# Define the S3 URI for your MP3 file
s3_uri = "s3://knowledge-base-us-west-2-707684582322/transcript/inputs/bedrock.mp3"

# Create the JSON structure
input_data = {
    "s3_uri": s3_uri
}

# Write to input.json file
with open('input.json', 'w', encoding='utf-8') as f:
    json.dump(input_data, f, ensure_ascii=False, indent=2)

print(f"Created input.json with S3 URI: {s3_uri}")

Created input.json with S3 URI: s3://knowledge-base-us-west-2-707684582322/transcript/inputs/bedrock.mp3


In [21]:
!aws s3 cp input.json s3://knowledge-base-us-west-2-707684582322/transcript/inputs/bedrock_input.json

upload: ./input.json to s3://knowledge-base-us-west-2-707684582322/transcript/inputs/bedrock_input.json


In [49]:
import boto3

sagemaker_client = boto3.client('sagemaker-runtime')

response = sagemaker_client.invoke_endpoint_async(
    EndpointName='whisper-async-endpoint',  # Your endpoint name
    InputLocation='s3://knowledge-base-us-west-2-707684582322/transcript/inputs/bedrock.mp3'  # S3 URI of input data
)

print(f"Response: {response}")

Response: {'ResponseMetadata': {'RequestId': '259352c7-8caf-495c-9dc5-05734b1b0ab4', 'HTTPStatusCode': 202, 'HTTPHeaders': {'x-amzn-requestid': '259352c7-8caf-495c-9dc5-05734b1b0ab4', 'x-amzn-sagemaker-outputlocation': 's3://knowledge-base-us-west-2-707684582322/transcript/outputs/3f49c884-670f-4a6d-bc7b-994e1af9d95a.out', 'date': 'Thu, 22 May 2025 15:14:09 GMT', 'content-type': 'application/json', 'content-length': '54', 'connection': 'keep-alive'}, 'RetryAttempts': 0}, 'OutputLocation': 's3://knowledge-base-us-west-2-707684582322/transcript/outputs/3f49c884-670f-4a6d-bc7b-994e1af9d95a.out', 'InferenceId': 'a5f5695c-f2fd-4944-a440-b9e47385feb3'}


### Production setup consideration: e.g. auto scaling, sns etc

https://github.com/aws/amazon-sagemaker-examples/blob/main/async-inference/Async-Inference-Walkthrough.ipynb